#### Data Preparation

In [115]:
df_matches = pd.read_csv("matches.csv")
df_deliveries = pd.read_csv("deliveries.csv")


In [116]:
print(df_deliveries.shape)
print(df_matches.shape)


(260920, 17)
(1095, 20)


In [117]:
# Date to Year for better EDA
df_matches['year'] = pd.to_datetime(matches['date']).dt.year

In [118]:
df_deliveries['id'] = df_deliveries['match_id']
df_del = df_deliveries.drop(columns='match_id')

In [119]:
(df_del.id.dtype)

dtype('int64')

In [120]:
df_matches.id.dtype

dtype('int64')

In [121]:
merged_df = pd.merge(df_del,df_matches, on='id',how='outer' )
merged_df.shape

(260920, 37)

In [122]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 260920 entries, 0 to 260919
Data columns (total 37 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   inning            260920 non-null  int64  
 1   batting_team      260920 non-null  object 
 2   bowling_team      260920 non-null  object 
 3   over              260920 non-null  int64  
 4   ball              260920 non-null  int64  
 5   batter            260920 non-null  object 
 6   bowler            260920 non-null  object 
 7   non_striker       260920 non-null  object 
 8   batsman_runs      260920 non-null  int64  
 9   extra_runs        260920 non-null  int64  
 10  total_runs        260920 non-null  int64  
 11  extras_type       14125 non-null   object 
 12  is_wicket         260920 non-null  int64  
 13  player_dismissed  12950 non-null   object 
 14  dismissal_kind    12950 non-null   object 
 15  fielder           9354 non-null    object 
 16  id                26

In [123]:
# merged_df.to_csv("merged_matches&devlieries.csv")

#### Data Quality

In [170]:
df = merged_df.copy()

In [171]:
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
other_cols = df.select_dtypes(exclude=['int64', 'float64', 'object', 'category']).columns.tolist()

summary_data = [
    ["Numeric", len(numeric_cols), ", ".join(numeric_cols) if numeric_cols else "None"],
    ["Categorical", len(categorical_cols), ", ".join(categorical_cols) if categorical_cols else "None"],
    ["Other", len(other_cols), ", ".join(other_cols) if other_cols else "None"]
]

column_summary = pd.DataFrame(summary_data, columns=["Column Type", "Count", "Columns"])
display(column_summary)

,Column Type,Count,Columns
0,Numeric,12,"inning, over, ball, batsman_runs, extra_runs, ..."
1,Categorical,25,"batting_team, bowling_team, batter, bowler, no..."
2,Other,0,None


In [172]:
# Check for Missing Values
print(df.shape)
df.isnull().sum().sort_values(ascending=False)

(260920, 37)


method              257274
fielder             251566
dismissal_kind      247970
player_dismissed    247970
extras_type         246795
city                 12397
result_margin         4124
player_of_match        490
winner                 490
target_runs            309
target_overs           309
result                   0
toss_decision            0
super_over               0
umpire1                  0
umpire2                  0
toss_winner              0
team2                    0
team1                    0
venue                    0
inning                   0
match_type               0
date                     0
batting_team             0
season                   0
id                       0
is_wicket                0
total_runs               0
extra_runs               0
batsman_runs             0
non_striker              0
bowler                   0
batter                   0
ball                     0
over                     0
bowling_team             0
year                     0
d

In [173]:
df.method.unique()

array([nan, 'D/L'], dtype=object)

In [174]:
# dropping method because we are not taking matches with no results due to some unseen circumstances 
# drop date as we are using year

df.drop(columns=['method','date','fielder','result_margin'], inplace=True)

In [175]:
df.shape

(260920, 33)

In [176]:
df['dismissal_kind'].value_counts(dropna=False)

NaN                      247970
caught                     8063
bowled                     2212
run out                    1114
lbw                         800
caught and bowled           367
stumped                     358
retired hurt                 15
hit wicket                   15
obstructing the field         3
retired out                   3
Name: dismissal_kind, dtype: int64

In [177]:
df['player_dismissed'].value_counts(dropna=False)

NaN            247970
RG Sharma         223
V Kohli           207
S Dhawan          193
KD Karthik        184
                ...  
BJ Haddin           1
SS Cottrell         1
JE Taylor           1
NL McCullum         1
Sunny Gupta         1
Name: player_dismissed, Length: 630, dtype: int64

In [178]:
df['extras_type'].value_counts(dropna=False)

NaN        246795
wides        8380
legbyes      4001
noballs      1069
byes          673
penalty         2
Name: extras_type, dtype: int64

In [179]:
df['batting_team'].unique()

array(['Kolkata Knight Riders', 'Royal Challengers Bangalore',
       'Chennai Super Kings', 'Kings XI Punjab', 'Rajasthan Royals',
       'Delhi Daredevils', 'Mumbai Indians', 'Deccan Chargers',
       'Kochi Tuskers Kerala', 'Pune Warriors', 'Sunrisers Hyderabad',
       'Rising Pune Supergiants', 'Gujarat Lions',
       'Rising Pune Supergiant', 'Delhi Capitals', 'Punjab Kings',
       'Lucknow Super Giants', 'Gujarat Titans',
       'Royal Challengers Bengaluru'], dtype=object)

In [180]:
df['team1'].unique()


array(['Royal Challengers Bangalore', 'Kings XI Punjab',
       'Delhi Daredevils', 'Mumbai Indians', 'Kolkata Knight Riders',
       'Rajasthan Royals', 'Deccan Chargers', 'Chennai Super Kings',
       'Kochi Tuskers Kerala', 'Pune Warriors', 'Sunrisers Hyderabad',
       'Gujarat Lions', 'Rising Pune Supergiants',
       'Rising Pune Supergiant', 'Delhi Capitals', 'Punjab Kings',
       'Lucknow Super Giants', 'Gujarat Titans',
       'Royal Challengers Bengaluru'], dtype=object)

In [181]:
df['city'].value_counts(dropna=False)

Mumbai            41742
Kolkata           21846
Delhi             21269
Chennai           20706
Hyderabad         18355
Bangalore         14907
Chandigarh        14476
Jaipur            13520
NaN               12397
Pune              12239
Abu Dhabi          8799
Ahmedabad          8553
Bengaluru          6917
Durban             3643
Visakhapatnam      3537
Lucknow            3283
Dubai              3149
Dharamsala         3141
Centurion          2866
Rajkot             2381
Sharjah            2355
Navi Mumbai        2188
Indore             1965
Johannesburg       1940
Cuttack            1695
Port Elizabeth     1677
Ranchi             1671
Cape Town          1539
Raipur             1431
Mohali             1235
Kochi              1155
Kanpur              921
Nagpur              742
Guwahati            739
Kimberley           726
East London         715
Bloemfontein        500
Name: city, dtype: int64

In [182]:
df_city_nan = df[df['city'].isna()]
print(df_city_nan.id.nunique())
print(df_city_nan.venue.unique())

51
['Sharjah Cricket Stadium' 'Dubai International Cricket Stadium']


In [183]:
# Replacing nan values with values that make sense for our dataset
df['dismissal_kind'].fillna('Not_Dismissed', inplace=True)
df['player_dismissed'].fillna('Not_Dismissed', inplace=True)
df['extras_type'].fillna('No_extras', inplace=True)
df['city'].fillna('UAE', inplace=True)

In [187]:
df_quality = df.dropna(subset=['player_of_match', 'winner', 'target_runs', 'target_overs']).reset_index(drop=True)


In [188]:
df_quality.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260430 entries, 0 to 260429
Data columns (total 33 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   inning            260430 non-null  int64  
 1   batting_team      260430 non-null  object 
 2   bowling_team      260430 non-null  object 
 3   over              260430 non-null  int64  
 4   ball              260430 non-null  int64  
 5   batter            260430 non-null  object 
 6   bowler            260430 non-null  object 
 7   non_striker       260430 non-null  object 
 8   batsman_runs      260430 non-null  int64  
 9   extra_runs        260430 non-null  int64  
 10  total_runs        260430 non-null  int64  
 11  extras_type       260430 non-null  object 
 12  is_wicket         260430 non-null  int64  
 13  player_dismissed  260430 non-null  object 
 14  dismissal_kind    260430 non-null  object 
 15  id                260430 non-null  int64  
 16  season            26

#### Data Cleaning

In [189]:
raw_clean_df = df_quality.copy()

In [194]:
raw_clean_df.columns

Index(['inning', 'batting_team', 'bowling_team', 'over', 'ball', 'batter',
       'bowler', 'non_striker', 'batsman_runs', 'extra_runs', 'total_runs',
       'extras_type', 'is_wicket', 'player_dismissed', 'dismissal_kind', 'id',
       'season', 'city', 'match_type', 'player_of_match', 'venue', 'team1',
       'team2', 'toss_winner', 'toss_decision', 'winner', 'result',
       'target_runs', 'target_overs', 'super_over', 'umpire1', 'umpire2',
       'year'],
      dtype='object')

In [190]:
teams_map = {'Royal Challengers Bangalore':'Bangalore'
              , 'Kings XI Punjab':'Punjab'
              , 'Delhi Daredevils':'Delhi'
              , 'Mumbai Indians':'Mumbai'
              , 'Kolkata Knight Riders':'Kolkata'
              , 'Rajasthan Royals':'Rajasthan'
              , 'Deccan Chargers':'Hyderabad'
              , 'Chennai Super Kings':'Chennai'
              , 'Kochi Tuskers Kerala':'Kerala'
              , 'Pune Warriors':'Pune'
              , 'Sunrisers Hyderabad':'Hyderabad'
              , 'Gujarat Lions':'Gujarat'
              , 'Rising Pune Supergiants':'Pune'
              , 'Rising Pune Supergiant':'Pune'
              , 'Delhi Capitals':'Delhi'
              , 'Punjab Kings':'Punjab'
              , 'Lucknow Super Giants':'Lucknow'
              , 'Gujarat Titans':'Gujarat'
              , 'Royal Challengers Bengaluru':'Bangalore'
}
for col in (raw_clean_df.columns):
    raw_clean_df[col] = raw_clean_df[col].apply(lambda x: teams_map.get(x, x))

In [191]:
venue_mapping = {
    'Eden Gardens, Kolkata': 'Eden Gardens',
    'MA Chidambaram Stadium, Chepauk': 'MA Chidambaram Stadium',
    'MA Chidambaram Stadium, Chepauk, Chennai': 'MA Chidambaram Stadium',
    'Wankhede Stadium, Mumbai': 'Wankhede Stadium',
    'Punjab Cricket Association IS Bindra Stadium, Mohali': 'Punjab Cricket Association Stadium',
    'Punjab Cricket Association IS Bindra Stadium': 'Punjab Cricket Association Stadium',
    'Punjab Cricket Association IS Bindra Stadium, Mohali, Chandigarh': 'Punjab Cricket Association Stadium',
    'Rajiv Gandhi International Stadium, Uppal': 'Rajiv Gandhi International Stadium',
    'Rajiv Gandhi International Stadium, Uppal, Hyderabad': 'Rajiv Gandhi International Stadium',
    'Arun Jaitley Stadium, Delhi': 'Arun Jaitley Stadium',
    'Dr DY Patil Sports Academy, Mumbai': 'Dr DY Patil Sports Academy',
    'Maharashtra Cricket Association Stadium, Pune': 'Maharashtra Cricket Association Stadium',
    'M Chinnaswamy Stadium, Bengaluru': 'M Chinnaswamy Stadium',
    'Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium, Visakhapatnam': 'Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium',
    'Sawai Mansingh Stadium, Jaipur': 'Sawai Mansingh Stadium',
    'Himachal Pradesh Cricket Association Stadium, Dharamsala': 'Himachal Pradesh Cricket Association Stadium'
}
for col in (raw_clean_df.columns):
    raw_clean_df[col] = raw_clean_df[col].apply(lambda x: venue_mapping.get(x, x))

In [199]:
print(f"Unique values in Team before mapping: {df_quality.team1.nunique()}")
print(f"Unique values in Team after mapping: {raw_clean_df.team1.nunique()}")
print("The reduction in unique Team values indicates that different categories have been properly mapped to unified values.\n")

print(f"Unique values in Venue before mapping: {df_quality.venue.nunique()}")
print(f"Unique values in Venue after mapping: {raw_clean_df.venue.nunique()}")
print("The decrease in unique Venue values confirms that varying category names have been correctly consolidated.")


Unique values in Team before mapping: 19
Unique values in Team after mapping: 12
The reduction in unique Team values indicates that different categories have been properly mapped to unified values.



Unique values in Venue before mapping: 58
Unique values in Venue after mapping: 43
The decrease in unique Venue values confirms that varying category names have been correctly consolidated.


In [202]:
final_cleaned_df = raw_clean_df.copy()

In [206]:
merged_df.shape

(260920, 37)

In [203]:
final_cleaned_df.shape

(260430, 33)

In [205]:
final_cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260430 entries, 0 to 260429
Data columns (total 33 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   inning            260430 non-null  int64  
 1   batting_team      260430 non-null  object 
 2   bowling_team      260430 non-null  object 
 3   over              260430 non-null  int64  
 4   ball              260430 non-null  int64  
 5   batter            260430 non-null  object 
 6   bowler            260430 non-null  object 
 7   non_striker       260430 non-null  object 
 8   batsman_runs      260430 non-null  int64  
 9   extra_runs        260430 non-null  int64  
 10  total_runs        260430 non-null  int64  
 11  extras_type       260430 non-null  object 
 12  is_wicket         260430 non-null  int64  
 13  player_dismissed  260430 non-null  object 
 14  dismissal_kind    260430 non-null  object 
 15  id                260430 non-null  int64  
 16  season            26